# Create Network Dataset

How to create network **Dataset**. 
**Add some description here**

In [ ]:
import ast, os
import pandas as pd
import networkx as nx

from pyincore import IncoreClient, DataService, Dataset, NetworkDataset, NetworkUtil

In [ ]:
client = IncoreClient()
datasvc = DataService(client)

### Create Graph file in CSV using local Link shapefile

In [ ]:
# get link dataset from data service
centerville_epn_link = '5b1fdc2db1cf3e336d7cecc9'
node_dataset = Dataset.from_data_service(centerville_epn_link, DataService(client))
df = node_dataset.get_dataframe_from_shapefile()

# get the field name from the node dataset dataframe
id_fldname = 'linknwid'
fromnode_fldname = 'fromnode'
tonode_fldname = 'tonode'

# save extracted fromnode, tonode information as a graph file 
file_name = "network_graph.csv"
try:
    df1 = df[[id_fldname, fromnode_fldname, tonode_fldname]]
    df1.to_csv(file_name, encoding='utf-8', index=False, header=['linkid', 'fromnode', 'tonode'])
    print(file_name, "successfully created.")
except:
    raise Exception("creating graph file failed.")

### Create NetworkX Graph using Link dataset in the code instead of reading the graph file

In [ ]:
# get node dataset from data service
centerville_epn_link = '5b1fdc2db1cf3e336d7cecc9'
fromnode_fldname = 'fromnode'
tonode_fldname = 'tonode'
link_dataset = Dataset.from_data_service(centerville_epn_link, DataService(client)).get_inventory_reader()
link_filepath = os.path.join(link_dataset.path, link_dataset.name + ".shp")
# the actual graph function in pyincore to directly create graph from the dataset and use
# instead of reading it from the graph file
g, node_coords = NetworkUtil.create_network_graph_from_link(link_filepath, fromnode_fldname, tonode_fldname, is_directed=True)
print(node_coords)
print(g)
nx.draw(g, node_coords)

### Create Link shapefile from Node and Graph dataset

In [ ]:
# this process can be done using the local shapefile, 
# instead of using the dataset from the incore data service.
# simply, provide the node_filename and graph_filename that are the file paths

# get node dataset from data service
centerville_epn_node = '5b1fdb50b1cf3e336d7cecb1'
node_dataset = Dataset.from_data_service(centerville_epn_node, DataService(client)).get_inventory_reader()
node_filename = os.path.join(node_dataset.path, node_dataset.name + ".shp")

# get graph dataset from data serivce
centerville_epn_network_graph = '6215681e45bb42602207cdb8'
graph_dataset = Dataset.from_data_service(centerville_epn_network_graph, DataService(client))
graph_filename = graph_dataset.get_file_path()

# create output link shapefile
id_field = 'nodenwid'
out_filename = 'network_links.shp'
NetworkUtil.build_link_by_node(node_filename, graph_filename, id_field, out_filename)

### Create Node shapefile and Graph file from Link dataset

In [ ]:
# this process can be done using the local shapefile, 
# instead of using the dataset from the incore data service.
# simply, provide the node_filename and graph_filename that are the file paths

# get link dataset from data service
centerville_epn_link = '5b1fdc2db1cf3e336d7cecc9'
link_dataset = Dataset.from_data_service(centerville_epn_link, DataService(client)).get_inventory_reader()
link_filename = os.path.join(link_dataset.path, link_dataset.name + ".shp")

# set input field names
link_id_field = 'linknwid'
fromnode_field = 'fromnode'
tonode_field = 'tonode'

# set output file names
out_node_filename = 'network_nodes.shp'
out_graph_filename = 'network_graph.csv'

# create output link shapefile and graph file
NetworkUtil.build_node_by_link(link_filename, link_id_field, fromnode_field, tonode_field, out_node_filename, out_graph_filename)

### Upload/Create Network Datasets to Data Services
This example shows how to create network dataset by uploading a link shapefile and a node shapefile

#### Prerequisite

- **meta data** is json text descrbing the network dataset component
- **node file** is a shapefile that contains node information of the network. 
- **link file** is a shapefile that contains the link information. This should include fromnode and tonode information.
- **graph file** is a csv file that contains the network relationship between the link and node.

In [ ]:
dataset_prop = {
  "title": "Test epn network",
  "dataType": "incore:epnNetwork",
  "format": "shp-network",
  "networkDataset": {
    "link": {
      "dataType": "ergo:powerLineTopo",
      "fileName": "epn_links.shp"
    },
    "node": {
      "dataType": "incore:epf",
      "fileName": "epn_nodes.shp"
    },
    "graph": {
      "dataType": "incore:networkGraph",
      "fileName": "network_graph.csv"
    }
  }
}

response = datasvc.create_dataset(dataset_prop)

if 'id' not in response:
    print("Failed to create a dataset in the data service")
else:
    dataset_id = response['id']

    print('network dataset is created with id ' + dataset_id)
    print('attching files to created dataset')

    file_dir = os.path.join(os.getcwd(), 'files')

    files = ['epn_links.shp',
             'epn_links.dbf',
             'epn_links.shx',
             'epn_links.prj',
             'epn_nodes.shp',
             'epn_nodes.dbf',
             'epn_nodes.shx',
             'epn_nodes.prj',
             'network_graph.csv']
    linkname = dataset_prop["networkDataset"]["link"]["fileName"]
    nodename = dataset_prop["networkDataset"]["node"]["fileName"]
    graphname = dataset_prop["networkDataset"]["graph"]["fileName"]

    response = datasvc.add_files_to_network_dataset(dataset_id, files, nodename, linkname, graphname)
    dataset = Dataset.from_data_service(response["id"], data_service=datasvc)